# 실습 3. CRM 고객 추천화 서비스
1. 데이터: Online Retail.csv
2. 추천화 서비스 구현 (Collaborate Filtering)
3. 특징: 유사도 사용 고객-고객, 아이템-아이템 Matrix 구현
4. 목표:
- Customer 기반 Collaborative filtering
- Item 기반 Collaborate filtering

Matrix를 만듬  
- pivot_table - index: customerId, columns: StockCode
- nan값을 0으로 0 이상이면 1값으로 변경
- cosine sim matrix
- Making Recommendations
- 유사한 고객들에게 사지 않은 제품을 추천해줌
    - A와 B가 유사 고객일 때 A가 사고 B가 사지않은 제품이 있을 때 B에게 A가 산 제품 추천

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Data 불러오기

In [2]:
df = pd.read_csv('../data/OnlineRetail.csv')
df.shape

(541909, 8)

## Data 확인

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 8:26,3.39,17850.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df.loc[df['CustomerID'].isna()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32,1.66,NaN,United Kingdom


## null이 포함된 행 제거

In [6]:
# subset= ['column'] column값이 null인 값만 제거
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


## Customer-Item Matrix

In [7]:
Customer_Item_df = pd.pivot_table(data = df, index = ['CustomerID'], columns='StockCode',values = 'Quantity',aggfunc = 'sum')
Customer_Item_df.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [8]:
Customer_Item_df= Customer_Item_df.applymap(lambda x: 1 if x > 0 else 0)
Customer_Item_df.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
Customer_Item_df.shape

(4372, 3684)

In [10]:
df['CustomerID'].nunique()

4372

In [11]:
df['StockCode'].nunique()

3684

## cosine_similarity

### Customer-Customer

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

Customer_Customer_df = pd.DataFrame(cosine_similarity(Customer_Item_df))
Customer_Customer_df.columns = Customer_Item_df.index
Customer_Customer_df['CustomerID'] = Customer_Item_df.index
Customer_Customer_df = Customer_Customer_df.set_index('CustomerID')
Customer_Customer_df.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038814,0.0,0.025876,0.136641,0.094742,...,0.0,0.0,0.054656,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027995,0.0,0.027995,0.118262,0.146427,...,0.0,0.0,0.118262,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.138314,0.0,0.030737,0.032461,0.144692,...,0.0,0.0,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031846,0.0,0.000000,0.000000,0.033315,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


In [13]:
Customer_Customer_df.shape

(4372, 4372)

In [14]:
Customer_Customer_df.loc[12350.0].sort_values(ascending=False)

CustomerID
12350.0    1.000000
17935.0    0.183340
12414.0    0.181902
12652.0    0.175035
12814.0    0.171499
             ...   
14869.0    0.000000
14870.0    0.000000
14873.0    0.000000
14875.0    0.000000
18287.0    0.000000
Name: 12350.0, Length: 4372, dtype: float64

In [16]:
Item_bought_by_A = set(Customer_Customer_df.loc[12350.0].iloc[Customer_Customer_df.loc[12350.0].to_numpy().nonzero()].index)
print(len(Item_bought_by_A))
Item_bought_by_A

1440


{16389.0,
 16392.0,
 16393.0,
 16395.0,
 16399.0,
 16401.0,
 16402.0,
 16403.0,
 16411.0,
 16413.0,
 16415.0,
 16418.0,
 16422.0,
 16426.0,
 16431.0,
 16434.0,
 16440.0,
 16442.0,
 16444.0,
 16460.0,
 16469.0,
 16470.0,
 16474.0,
 16477.0,
 16480.0,
 16483.0,
 16495.0,
 16496.0,
 16497.0,
 16499.0,
 16520.0,
 16531.0,
 16533.0,
 16536.0,
 16549.0,
 16553.0,
 16556.0,
 16571.0,
 16602.0,
 16603.0,
 16609.0,
 16612.0,
 16618.0,
 16629.0,
 16637.0,
 16644.0,
 16648.0,
 16650.0,
 16652.0,
 16657.0,
 16669.0,
 16670.0,
 16674.0,
 16678.0,
 16680.0,
 16682.0,
 16684.0,
 16686.0,
 16692.0,
 16701.0,
 16705.0,
 16706.0,
 16710.0,
 16712.0,
 16713.0,
 16726.0,
 16727.0,
 16728.0,
 16735.0,
 16743.0,
 16744.0,
 16745.0,
 16746.0,
 16750.0,
 16754.0,
 16755.0,
 16758.0,
 16759.0,
 16763.0,
 16764.0,
 16770.0,
 16771.0,
 16779.0,
 16782.0,
 16788.0,
 16790.0,
 16794.0,
 16800.0,
 16805.0,
 16813.0,
 16818.0,
 16828.0,
 16834.0,
 16843.0,
 16846.0,
 16848.0,
 16869.0,
 16873.0,
 16875.0,
 16877.0,


In [17]:
Customer_Customer_df.loc[17935.0].iloc[Customer_Customer_df.loc[17935.0].to_numpy().nonzero()].index

Index([12348.0, 12349.0, 12350.0, 12352.0, 12356.0, 12358.0, 12359.0, 12360.0,
       12361.0, 12362.0,
       ...
       18226.0, 18235.0, 18241.0, 18242.0, 18257.0, 18262.0, 18269.0, 18272.0,
       18283.0, 18287.0],
      dtype='object', name='CustomerID', length=1866)

In [18]:
Item_bought_by_B = set(Customer_Customer_df.loc[17935.0].iloc[Customer_Customer_df.loc[17935.0].to_numpy().nonzero()].index)
len(Item_bought_by_B)

1866

In [19]:
from collections import Counter

Item_to_recommend_to_B = Item_bought_by_A - Item_bought_by_B
len(Item_to_recommend_to_B)

322

In [20]:
df.loc[df['CustomerID'].isin(Item_to_recommend_to_B), ['StockCode', 
                                                       'Description']].drop_duplicates().set_index('StockCode').head()

,Description
StockCode,
10002,INFLATABLE POLITICAL GLOBE
21912,VINTAGE SNAKES & LADDERS
21832,CHOCOLATE CALCULATOR
22411,JUMBO SHOPPER VINTAGE RED PAISLEY
22379,RECYCLING BAG RETROSPOT


### Item-Item

In [21]:
Item_Item_df = pd.DataFrame(cosine_similarity(Customer_Item_df.T))
Item_Item_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3674,3675,3676,3677,3678,3679,3680,3681,3682,3683
0,1.000000,0.0,0.094868,0.091287,0.0,0.000000,0.090351,0.063246,0.098907,0.095346,...,0.0,0.0,0.0,0.029361,0.0,0.0,0.0,0.059423,0.0,0.070057
1,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.032774,0.045883,0.047836,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.017244,0.0,0.000000
2,0.094868,0.0,1.000000,0.115470,0.0,0.000000,0.057143,0.060000,0.041703,0.060302,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.075165,0.0,0.000000
3,0.091287,0.0,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
4,0.000000,0.0,0.000000,0.000000,1.0,0.447214,0.063888,0.044721,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000


In [22]:
Item_Item_df.columns = Customer_Item_df.T.index
Item_Item_df['StockCode'] = Customer_Item_df.T.index
Item_Item_df = Item_Item_df.set_index('StockCode')
Item_Item_df.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.0,0.094868,0.091287,0.0,0.000000,0.090351,0.063246,0.098907,0.095346,...,0.0,0.0,0.0,0.029361,0.0,0.0,0.0,0.059423,0.0,0.070057
10080,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.032774,0.045883,0.047836,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.017244,0.0,0.000000
10120,0.094868,0.0,1.000000,0.115470,0.0,0.000000,0.057143,0.060000,0.041703,0.060302,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.075165,0.0,0.000000
10123C,0.091287,0.0,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
10124A,0.000000,0.0,0.000000,0.000000,1.0,0.447214,0.063888,0.044721,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000


## StockCode가 0124G와 관련성 높은 순으로 10개의 상품

In [26]:
top_10_similar_items = list(Item_Item_df.loc['10124G'].sort_values(ascending=False).iloc[:10].index)

In [24]:
top_10_similar_items

['10124G',
 '10124A',
 '79190D',
 '22346',
 '47351B',
 '82011A',
 '84466',
 '85095',
 '22878',
 '22265']

In [25]:
df.loc[df['StockCode'].isin(top_10_similar_items), ['StockCode', 
                                                    'Description']].drop_duplicates().set_index('StockCode').loc[top_10_similar_items]

,Description
StockCode,
10124G,ARMY CAMO BOOKCOVER TAPE
10124A,SPOTS ON RED BOOKCOVER TAPE
79190D,RETRO PLASTIC DAISY TRAY
22346,PARTY PIZZA DISH GREEN POLKADOT
47351B,BLUE VOILE LAMPSHADE
82011A,"BATHROOM SCALES, TROPICAL BEACH"
84466,TOP SECRET PEN SET
85095,THREE CANVAS LUGGAGE TAGS
22878,NUMBER TILE COTTAGE GARDEN No
